In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from datetime import datetime as dt
import seaborn as sns


In [94]:
df_consultas = pd.read_csv('consultas_final.csv', sep = ',')
df_avaliacoes = pd.read_csv('avaliacoes_final.csv', sep = ',')
df_clinicas = pd.read_csv('clinicas_final.csv', sep = ',')
df_medicos = pd.read_csv('medicos_final.csv', sep = ';')
df_pacientes = pd.read_csv('pacientes_final.csv', sep = ',')

In [95]:
df_consultas.head(10)

,id_consulta,id_paciente,id_medico,id_clinica,especialidade,data_hora_agendada,data_hora_inicio,status
0,1,82,15,4,Cardiologia,2023-09-27 23:46,2023-09-28 00:16,Realizada
1,2,18,95,14,Cardiologia,2023-09-04 09:05,2023-09-04 13:05,Realizada
2,3,55,5,4,Pediatria,2023-02-04 02:43,2023-02-04 02:58,Realizada
3,4,78,4,72,Ortopedia,2023-03-14 09:28,2023-03-14 13:28,Realizada
4,5,58,76,36,Pediatria,2023-10-22 16:54,2023-10-22 16:54,Cancelada
5,6,21,90,55,Dermatologia,2023-05-04 21:09,2023-05-04 21:39,Realizada
6,7,98,44,14,Cardiologia,2023-02-03 18:25,2023-02-03 20:25,Realizada
7,8,45,78,34,Ortopedia,2023-10-21 20:47,2023-10-21 20:47,Cancelada
8,9,69,16,49,Ortopedia,2023-01-29 16:33,2023-01-29 20:33,Realizada
9,10,81,80,47,Ginecologia,2023-07-30 04:58,2023-07-30 05:13,Cancelada


In [96]:



medicos.rename(columns={'nome': 'nome_medico', 'especialidade': 'especialidade_medico'}, inplace=True)
clinicas.rename(columns={'nome': 'nome_clinica'}, inplace=True)

# Merge dos dataframes
df = pd.merge(consultas, medicos, on='id_medico', how='left')
df = pd.merge(df, clinicas, on='id_clinica', how='left')
df = pd.merge(df, avaliacoes, on='id_consulta', how='left')
df = pd.merge(df, pacientes, on='id_paciente', how='left')

# 1. Tratar os dados e calcular o tempo de espera
df['data_hora_agendada'] = pd.to_datetime(df['data_hora_agendada'])
df['data_hora_inicio'] = pd.to_datetime(df['data_hora_inicio'])
df['tempo_de_espera'] = (df['data_hora_inicio'] - df['data_hora_agendada']).dt.total_seconds() / 60  # em minutos

# Filtrar status "Realizada" e tempo de espera não negativo
df_realizadas = df[df['status'] == 'Realizada'].copy()
df_realizadas = df_realizadas[df_realizadas['tempo_de_espera'] >= 0]

print("Visualização dos dados após o tratamento inicial:")
print(df_realizadas[['id_consulta', 'especialidade', 'tempo_de_espera', 'status']].head())


# 2. Identificar outliers usando quartis
Q1 = df_realizadas['tempo_de_espera'].quantile(0.25)
Q2 = df_realizadas['tempo_de_espera'].quantile(0.50)
Q3 = df_realizadas['tempo_de_espera'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR
limite_inferior = Q1 - 1.5 * IQR

outliers = df_realizadas[(df_realizadas['tempo_de_espera'] > limite_superior) | (df_realizadas['tempo_de_espera'] < limite_inferior)]

print("\n--- Análise de Outliers ---")
print(f"Q1 (25º percentil): {Q1:.2f} minutos")
print(f"Q2 (Mediana, 50º percentil): {Q2:.2f} minutos")
print(f"Q3 (75º percentil): {Q3:.2f} minutos")
print(f"Número de outliers identificados: {len(outliers)}")


# 3. Comparar média e mediana por especialidade, médico e clínica
print("\n--- Média e Mediana do Tempo de Espera ---")

# Por especialidade (agora funciona pois 'especialidade' não tem mais conflito)
media_mediana_especialidade = df_realizadas.groupby('especialidade')['tempo_de_espera'].agg(['mean', 'median']).sort_values(by='mean', ascending=False)
print("\nPor Especialidade:")
print(media_mediana_especialidade)

# Por médico
media_mediana_medico = df_realizadas.groupby('nome_medico')['tempo_de_espera'].agg(['mean', 'median']).sort_values(by='mean', ascending=False)
print("\nPor Médico (Top 5 com maior tempo médio de espera):")
print(media_mediana_medico.head())

# Por clínica
media_mediana_clinica = df_realizadas.groupby('nome_clinica')['tempo_de_espera'].agg(['mean', 'median']).sort_values(by='mean', ascending=False)
print("\nPor Clínica:")
print(media_mediana_clinica)


# 4. Explorar correlações entre tempo de espera e nota de satisfação
correlacao = df_realizadas[['tempo_de_espera', 'nota_satisfacao']].corr()

print("\n--- Correlação entre Tempo de Espera e Nota de Satisfação ---")
print(correlacao)

Visualização dos dados após o tratamento inicial:
   id_consulta especialidade  tempo_de_espera     status
0            1   Cardiologia             30.0  Realizada
1            2   Cardiologia            240.0  Realizada
2            3     Pediatria             15.0  Realizada
3            4     Ortopedia            240.0  Realizada
5            6  Dermatologia             30.0  Realizada

--- Análise de Outliers ---
Q1 (25º percentil): 10.00 minutos
Q2 (Mediana, 50º percentil): 30.00 minutos
Q3 (75º percentil): 180.00 minutos
Número de outliers identificados: 0

--- Média e Mediana do Tempo de Espera ---

Por Especialidade:
                    mean  median
especialidade                   
Dermatologia   98.548851    60.0
Pediatria      98.082287    30.0
Neurologia     95.198675    60.0
Ginecologia    95.017422    30.0
Ortopedia      94.656992    30.0
Cardiologia    93.611842    45.0

Por Médico (Top 5 com maior tempo médio de espera):
                             mean  median
nome_med

In [97]:
#------------------------------------------------------- NOVA PEDIDA ------------------------------------------------------------------------#

In [98]:

df_merged = pd.merge(consultas, avaliacoes, on='id_consulta', how='inner')
df_merged = pd.merge(df_merged, pacientes, on='id_paciente', how='inner')

# 2. Definir pacientes insatisfeitos (nota de satisfação 1 ou 2)
df_insatisfeitos = df_merged[df_merged['nota_satisfacao'].isin([1, 2])].copy()

# 3. Criar faixas etárias para agrupar os pacientes
bins = [0, 17, 30, 45, 60, 100]
labels = ['0-17', '18-30', '31-45', '46-60', '61+']
df_insatisfeitos['faixa_etaria'] = pd.cut(df_insatisfeitos['idade'], bins=bins, labels=labels, right=False)

# 4. Análise: Pacientes insatisfeitos por Cidade e Especialidade
insatisfacao_cidade_especialidade = df_insatisfeitos.groupby(['cidade', 'especialidade']).size().reset_index(name='n_insatisfeitos')
insatisfacao_cidade_especialidade = insatisfacao_cidade_especialidade.sort_values(by='n_insatisfeitos', ascending=False)

print("--- Pacientes Insatisfeitos por Cidade e Especialidade (Top 10) ---")
print(insatisfacao_cidade_especialidade.head(10))

# 5. Análise: Pacientes insatisfeitos por Faixa Etária e Especialidade
insatisfacao_idade_especialidade = df_insatisfeitos.groupby(['faixa_etaria', 'especialidade']).size().reset_index(name='n_insatisfeitos')
insatisfacao_idade_especialidade = insatisfacao_idade_especialidade.sort_values(by='n_insatisfeitos', ascending=False)

print("\n--- Pacientes Insatisfeitos por Faixa Etária e Especialidade (Top 10) ---")
print(insatisfacao_idade_especialidade.head(10))

--- Pacientes Insatisfeitos por Cidade e Especialidade (Top 10) ---
            cidade especialidade  n_insatisfeitos
3         Brasília   Cardiologia                4
27        Salvador     Ortopedia                4
28        Salvador     Pediatria                3
22          Recife     Pediatria                2
17          Manaus     Pediatria                2
4         Brasília  Dermatologia                2
5         Brasília   Ginecologia                2
23  Rio de Janeiro   Cardiologia                2
21    Porto Alegre     Pediatria                2
24  Rio de Janeiro     Pediatria                2

--- Pacientes Insatisfeitos por Faixa Etária e Especialidade (Top 10) ---
   faixa_etaria especialidade  n_insatisfeitos
28          61+     Ortopedia                5
11        18-30     Pediatria                4
23        46-60     Pediatria                4
24          61+   Cardiologia                4
12        31-45   Cardiologia                3
5          0-17     Pedia

C:\Users\tatud\AppData\Local\Temp\ipykernel_17548\1045610251.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  insatisfacao_idade_especialidade = df_insatisfeitos.groupby(['faixa_etaria', 'especialidade']).size().reset_index(name='n_insatisfeitos')


In [99]:


clinicas.rename(columns={'nome': 'nome_clinica'}, inplace=True)

# 1. Unir os dataframes usando merge (left join para manter todas as clínicas)
df_analise = pd.merge(clinicas, consultas, on='id_clinica', how='left')
df_analise = pd.merge(df_analise, avaliacoes, on='id_consulta', how='left')

# 2. Filtrar apenas por consultas com status 'Realizada'
df_realizadas = df_analise[df_analise['status'] == 'Realizada'].copy()

# 3. Agrupar por clínica para calcular o desempenho

desempenho_clinicas = df_realizadas.groupby(['id_clinica', 'nome_clinica', 'capacidade_diaria']).agg(
    total_atendimentos=('id_consulta', 'count'),
    media_satisfacao=('nota_satisfacao', 'mean')
).reset_index()

# Arredondar a média para melhor visualização
desempenho_clinicas['media_satisfacao'] = desempenho_clinicas['media_satisfacao'].round(2)

# 4. Ordenar por capacidade para facilitar a comparação
desempenho_clinicas = desempenho_clinicas.sort_values(by='capacidade_diaria', ascending=False)


print("--- Análise de Desempenho das Clínicas por Capacidade ---")
print(desempenho_clinicas.head(10))

# Calcular e exibir as correlações para uma resposta direta
correlacao_capacidade_volume = desempenho_clinicas[['capacidade_diaria', 'total_atendimentos']]
correlacao_capacidade_satisfacao = desempenho_clinicas[['capacidade_diaria', 'media_satisfacao']]



--- Análise de Desempenho das Clínicas por Capacidade ---
    id_clinica       nome_clinica  capacidade_diaria  total_atendimentos  \
19          20      Clínica Vital                100                  44   
26          27     Clínica Renova                 98                  41   
44          45  Clínica São Lucas                 98                  24   
78          79        Saúde Total                 97                  39   
79          80  Clínica Nova Vida                 97                  34   
32          33  Clínica São Lucas                 96                  32   
42          43  Clínica Esperança                 94                  33   
11          12  Clínica Horizonte                 94                  46   
4            5  Clínica São Lucas                 92                  33   
53          54  Clínica Esperança                 89                  24   

    media_satisfacao  
19               3.0  
26               3.0  
44               NaN  
78           

In [100]:

df_analise = pd.merge(consultas, pacientes, on='id_paciente', how='inner')
# Calcular o tempo de espera em minutos
df_analise['data_hora_agendada'] = pd.to_datetime(df_analise['data_hora_agendada'])
df_analise['data_hora_inicio'] = pd.to_datetime(df_analise['data_hora_inicio'])
df_analise['tempo_de_espera'] = (df_analise['data_hora_inicio'] - df_analise['data_hora_agendada']).dt.total_seconds() / 60


df_realizadas = df_analise[(df_analise['status'] == 'Realizada') & (df_analise['tempo_de_espera'] >= 0)].copy()

# Agrupar por 'plano_saude' e calcular as estatísticas do tempo de espera
analise_plano_saude = df_realizadas.groupby('plano_saude')['tempo_de_espera'].agg(['mean', 'median', 'count'])

analise_plano_saude.columns = ['Tempo Médio (min)', 'Mediana (min)', 'Nº de Atendimentos']
# Exibir o resultado
print("Análise do Tempo de Espera por Plano de Saúde")
print(analise_plano_saude)


Análise do Tempo de Espera por Plano de Saúde
             Tempo Médio (min)  Mediana (min)  Nº de Atendimentos
plano_saude                                                      
Não                  95.679535           60.0                1891
Sim                  95.583546           30.0                1568


In [101]:


df_analise = pd.merge(consultas, avaliacoes, on='id_consulta', how='left')

# 2. Agrupar por especialidade para calcular as métricas
analise_especialidade = df_analise.groupby('especialidade').agg(
    # Calcular a média de satisfação (NaNs são ignorados automaticamente)
    media_satisfacao=('nota_satisfacao', 'mean'),
   
    taxa_cancelamento_percentual=('status', lambda x: (x == 'Cancelada').sum() / x.count() * 100),
    
    total_consultas=('id_consulta', 'count')
).reset_index()

# Arredondar os valores para melhor visualização
analise_especialidade['media_satisfacao'] = analise_especialidade['media_satisfacao'].round(2)
analise_especialidade['taxa_cancelamento_percentual'] = analise_especialidade['taxa_cancelamento_percentual'].round(2)


analise_especialidade = analise_especialidade.sort_values(
    by=['taxa_cancelamento_percentual', 'media_satisfacao'],
    ascending=[False, True]
)

# Exibir o resultado final
print("--- Análise de Cancelamento e Satisfação por Especialidade ---")
print(analise_especialidade)

--- Análise de Cancelamento e Satisfação por Especialidade ---
  especialidade  media_satisfacao  taxa_cancelamento_percentual  \
1  Dermatologia              2.44                         21.76   
4     Ortopedia              3.14                         20.58   
0   Cardiologia              2.75                         20.36   
5     Pediatria              2.79                         19.96   
3    Neurologia              3.36                         19.32   
2   Ginecologia              2.71                         18.18   

   total_consultas  
1              501  
4             1113  
0             1110  
5             1022  
3              440  
2              814  
